In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import mysql.connector as sql
import csv
from PIL import Image, ImageTk
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Login Window

In [2]:
class LoginWindow:
    def __init__(self, root, on_successful_login):
        self.root = root
        self.root.title("Login to Green Grid Data")
        self.on_successful_login = on_successful_login

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg_LOGIN.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self.root, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        # Create the login frame
        self.login_frame = tk.Frame(self.root, bg='white')
        self.login_frame.pack(padx=10, pady=175)

        # Username field
        tk.Label(self.login_frame, text="Username:", bg='white').grid(row=0, column=0, padx=5, pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.grid(row=0, column=1, padx=5, pady=5)

        # Password field
        tk.Label(self.login_frame, text="Password:", bg='white').grid(row=1, column=0, padx=5, pady=5)
        self.password_entry = tk.Entry(self.login_frame, show="*")
        self.password_entry.grid(row=1, column=1, padx=5, pady=5)

        # Login button
        tk.Button(self.login_frame, text="Login", command=self.attempt_login).grid(row=2, column=0, columnspan=2, pady=10)

    def attempt_login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Read credentials from CSV file
        credentials = {}
        try:
            with open("credentials.csv", "r") as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    credentials[row["user"]] = row["password"]
        except FileNotFoundError:
            messagebox.showerror("Error", "credentials.csv file not found.")
            self.root.quit()
            return
        except Exception as e:
            messagebox.showerror("Error", f"Failed to read credentials: {e}")
            self.root.quit()
            return

        # Check if the username and password match
        if username in credentials and credentials[username] == password:
            self.login_frame.destroy()
            self.on_successful_login(username, password)
        else:
            messagebox.showerror("Login Failed", "Invalid username or password.")

# Main App Window

In [3]:
class GreenGridApp:
    def __init__(self, root, username, password, show_login):
        self.root = root
        self.root.title("Green Grid Data")
        self.show_login = show_login

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self.root, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        # Attempt to connect to the MySQL database using the provided credentials
        try:
            self.mydb = sql.connect(
                host="localhost",
                user="root",
                passwd="root",
                database="green-grid",
                port="3306"
            )
            self.mycursor = self.mydb.cursor()
        except sql.Error as e:
            messagebox.showerror("Database Error", f"Failed to connect to database: {e}")
            self.root.quit()
            return

        # Create the main frame and label
        self.main_frame = tk.Frame(self.root, bg='white')
        self.main_frame.pack(padx=10, pady=150)

        tk.Label(self.main_frame, text="Select a query:", bg='white').pack()

        # Add buttons for each query from output.ipynb
        tk.Button(self.main_frame, text="Peak Hour Usage per Meter", command=self.query1).pack(fill='x')
        tk.Button(self.main_frame, text="Zone-wise Total Energy Consumed", command=self.query2).pack(fill='x')
        tk.Button(self.main_frame, text="Billing Summaries per Customer", command=self.query3).pack(fill='x')

        # Add logout button
        tk.Button(self.main_frame, text="Logout", command=self.logout).pack(fill='x', pady=5)

        # Initialize and play background music
        try:
            pygame.mixer.init()
            pygame.mixer.music.load("background_music.mp3")
            pygame.mixer.music.set_volume(0.1)  # volume
            pygame.mixer.music.play(-1)  # loop indefinitely
        except Exception as e:
            messagebox.showerror("Music Error", f"Failed to load or play music: {e}")

        # Set window close protocol
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def logout(self):
        # Stop the music before logging out
        try:
            pygame.mixer.music.stop()
        except:
            pass
        self.close_connection()
        self.main_frame.destroy()
        self.show_login()

    def on_close(self):
        # Stop the music and close the application
        try:
            pygame.mixer.music.stop()
        except:
            pass
        self.close_connection()
        self.root.destroy()

    def show_main_window(self):
        self.root.deiconify()

    # Query 1: Peak Hour Usage per Meter
    def query1(self):
        self.root.withdraw()
        title = "Peak Hour Usage per Meter"
        sql_query = '''
            WITH ranked_usage AS (
              SELECT
                meter_id,
                HOUR(reading_time) AS hour_of_day,
                SUM(units_consumed) AS total_units,
                ROW_NUMBER() OVER (PARTITION BY meter_id ORDER BY SUM(units_consumed) DESC) AS rn
              FROM
                consumption_logs
              GROUP BY
                meter_id, hour_of_day
            )
            SELECT
              meter_id,
              hour_of_day,
              total_units
            FROM
              ranked_usage
            WHERE
              rn = 1;
        '''
        columns = ["Meter ID", "Hour of Day", "Total Units"]
        widths = [10, 15, 15]
        self.execute_query(title, sql_query, columns, widths)

    # Query 2: Zone-wise Total Energy Consumed
    def query2(self):
        self.root.withdraw()
        title = "Zone-wise Total Energy Consumed"
        sql_query = '''
            SELECT
                z.zone_id,
                z.zone_name,
                SUM(cl.units_consumed) AS total_units_consumed
            FROM
                consumption_logs cl
            JOIN
                meters m ON cl.meter_id = m.meter_id
            JOIN
                customers c ON m.customer_id = c.customer_id
            JOIN
                zones z ON c.zone_id = z.zone_id
            GROUP BY
                z.zone_id, z.zone_name
            ORDER BY
                z.zone_id;
        '''
        columns = ["Zone ID", "Zone Name", "Total Units Consumed"]
        widths = [10, 15, 20]
        self.execute_query(title, sql_query, columns, widths)

    # Query 3: Billing Summaries per Customer
    def query3(self):
        self.root.withdraw()
        title = "Billing Summaries per Customer"
        sql_query = '''
            SELECT
                c.customer_id,
                c.customer_name,
                COUNT(b.bill_id) AS total_bills,
                SUM(b.amount) AS total_billed_amount,
                SUM(CASE WHEN b.payment_status = 'unpaid' THEN 1 ELSE 0 END) AS unpaid_bills
            FROM
                billing b
            JOIN
                meters m ON b.meter_id = m.meter_id
            JOIN
                customers c ON m.customer_id = c.customer_id
            GROUP BY
                c.customer_id, c.customer_name
            ORDER BY
                customer_id;
        '''
        columns = ["Customer ID", "Customer Name", "Total Bills", "Total Billed Amount", "Unpaid Bills"]
        widths = [15, 30, 15, 20, 15]
        self.execute_query(title, sql_query, columns, widths)

    def execute_query(self, title, sql_query, columns, widths):
        try:
            self.mycursor.execute(sql_query)
            result = self.mycursor.fetchall()
            ResultsWindow(self, title, columns, result, widths)
        except sql.Error as e:
            messagebox.showerror("Query Error", f"Failed to execute query: {e}")
            self.show_main_window()

    def close_connection(self):
        if hasattr(self, 'mydb') and self.mydb.is_connected():
            self.mydb.close()
            self.mycursor.close()

# Result Window

In [4]:
class ResultsWindow(tk.Toplevel):
    def __init__(self, main_app, title, columns, data, widths):
        tk.Toplevel.__init__(self)
        self.main_app = main_app
        self.title(title)
        self.geometry("1280x720")

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        if not data:
            tk.Label(self, text="No results found.", bg='white').pack(pady=20)
        else:
            tree_frame = tk.Frame(self, bg='white')
            tree_frame.pack(expand=True, fill='both')

            tree = ttk.Treeview(tree_frame, columns=columns, show='headings')
            for col, width in zip(columns, widths):
                tree.heading(col, text=col)
                tree.column(col, width=width * 10)

            for row in data:
                tree.insert('', 'end', values=row)

            scrollbar = tk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=scrollbar.set)
            tree.pack(side='left', expand=True, fill='both')
            scrollbar.pack(side='right', fill='y')

        back_button = tk.Button(self, text="Back", command=self.go_back)
        back_button.pack(pady=10)

        self.protocol("WM_DELETE_WINDOW", self.go_back)

    def go_back(self):
        self.destroy()
        self.main_app.show_main_window()

# Main Function

In [5]:
def main():
    root = tk.Tk()
    root.geometry("1280x720")  # Set default window size

    def show_login():
        for widget in root.winfo_children():
            widget.destroy()
        LoginWindow(root, lambda username, password: GreenGridApp(root, username, password, show_login))
        root.protocol("WM_DELETE_WINDOW", root.destroy)

    # Start with the login window
    show_login()

    root.mainloop()

In [6]:
if __name__ == "__main__":
    main()